# LAB4 - Evaluating entity linking

This notebook describes evaluation for the task of entity linking. It covers the following aspects:
1. Aggregating scores
2. Evaluation procedure
3. Example
4. Example in code

### 1. Aggregating scores

The correctness of an entity linking system is measured in terms of precision, recall, and F1-score. 

Previously for the task of sentiment classification, we have seen that the precision, recall, and the F1-score were computed **per class**. This makes sense for tasks such as sentiment classification, because the number of classes is small and known in advance. For example, the classes in sentiment classification are: negative, neutral, and positive.

If we apply the same logic to the task of entity linking, we get a huge number of classes. One class is for example the JFK airport, another one is the president JFK, another one is Paris in France, etc. The total number of classes is in the range of millions and changes when the knowledge base changes: for example, if Wikipedia adds 10,000 new entities tomorrow, the number of classes increases by 10,000. In addition, there is one special class in this evaluation: the class of NIL entities, which covers all entities for which there is no link in the knowledge base we use. 

While computing per-class F1-score is possible for entity linking too and is sometimes done, it is  more common to use a different procedure, namely to score our entity linking systems **per occurrence of an entity mention**. Likewise, we  aggregate the F1-score per entity mention occurrence in this course. We explain the procedure in more detail in part 2 of this notebook.

### 2. Evaluation procedure
Similar to evaluations of classes, we create a confusion matrix across system and gold performance but in this case for entity mentions. For each of the mentions in text, we compare the system decision against the gold data:
* If the system chooses entity X and the gold entity is also X, and X is not a NIL entity, then we count a *true positive (TP)*
* If the system chooses entity X, but the gold entity is Y, then we count a *false positive (FP)* and a *false negative (FN)*
* If the system opts for a NIL entity and the gold entity is X, then we count a *false negative (FN)*
* If the system opts for an entity X but the gold entity is NIL, then we count a *false positive (FP)*
* If the system and the gold entities are both NIL, then we count a *true negative (TN)* - however, true negatives are not considered when computing the precision, recall, and F1-score.

Afterwards, we use these numbers for TP, FP, and FN, to compute precision, recall, and F1-score in the usual way:

* `precision=TP/(TP+FP)` -> From the decisions made by the system, how many were true
* `recall=TP/(TP+FN)` -> From the gold entities, how many were found correctly by the system
* `f1=2*precision*recall/(precision+recall)` -> compute a harmonic mean between precision and recall, called F1-score

Note that precision, recall, and F1-score would all be the same in case all entities in the system output and the gold output are not NIL entities.

### 3. Example

For the example sentence we used in the introductory notebook:

"_JetBlue_ begins direct service between _Barnstable Airport_ and _JFK_."

let's say that a system made the following decisions:
* "JetBlue" means http://dbpedia.org/resource/JetBlue (true positive)
* "Barnstable Airport" means http://dbpedia.org/resource/Barnstable,_Massachusetts (false positive)
* "JFK" means http://dbpedia.org/resource/John_F._Kennedy (false positive and false negative)

Then, we have in total: `TP=1, FP=2, FN=1`. 

The resulting precision is `1/3=0.33` and the resulting recall is `1/2=0.5`. 

The F1-score of this system on this sentence would be `0.40`. 

### 4. Example in code

Now we provide a code for this scenario. Note that for simplicity we assume that the entity recognition by the system is perfect. Also, we use a simple format of the gold and the system output as a list, in practice this requires some more preprocessing. 

In [2]:
def evaluate_entity_linking(system_decisions, gold_decisions):
    """
    Compute precision, recall, and F1-score by comparing two paired lists of: system decisions and gold data decisions.
    """
    tp=0
    fp=0
    fn=0

    for mention_id in range(num_entities):
        gold_entity=gold_decisions[mention_id]
        system_entity=system_decisions[mention_id]
        if gold_entity=='NIL' and system_entity=='NIL': continue
        if gold_entity==system_entity:
            tp+=1
        else:
            if gold_entity!='NIL':
                fn+=1
            if system_entity!='NIL':
                fp+=1

    print('TP: %d; \nFP: %d, \nFN: %d' % (tp, fp, fn))            

    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=2*precision*recall/(precision+recall)
    
    return precision, recall, f1

In [3]:
text="JetBlue begins direct service between Barnstable Airport and JFK."

gold_decisions=['http://dbpedia.org/resource/JetBlue', 
                'NIL',
                'http://dbpedia.org/resource/John_F._Kennedy_International_Airport']
system_decisions=['http://dbpedia.org/resource/JetBlue', 
                  'http://dbpedia.org/resource/Barnstable,_Massachusetts',
                 'http://dbpedia.org/resource/John_F._Kennedy']

num_entities=len(gold_decisions)

precision, recall, f1 = evaluate_entity_linking(system_decisions, gold_decisions)

print('Precision: %.2f, \nRecall: %.2f, \nF1-score: %.2f' % (precision, recall, f1))

TP: 1; 
FP: 2, 
FN: 1
Precision: 0.33, 
Recall: 0.50, 
F1-score: 0.40
